# Sloan Digital Sky Survey Classification
## Classification of Galaxies, Stars and Quasars based on the RD14 from the SDSS

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

SMALL_SIZE = 10
MEDIUM_SIZE = 12

plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=MEDIUM_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rcParams['figure.dpi']=150

In [2]:
#sdss_df = pd.read_csv('Skyserver_SQL2_27_2018 6_51_39 PM.csv', skiprows=1)
sdss_df = pd.read_csv('data/Skyserver_SQL2_27_2018 6_51_39 PM.csv', skiprows=0)

The dataset has 10000 examples, 17 feature columns and 1 target column. 8 of the 17 features are 64 bit integers, 1 feature is an unsigned 64 bit integer, 8 are 64 bit floats and the target column is of the type object. 

In [3]:
sdss_df.describe()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,redshift,plate,mjd,fiberid
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.237650e+18,175.529987,14.836148,18.619355,17.371931,16.840963,16.583579,16.422833,981.034800,301.0,3.648700,302.380100,1.645022e+18,0.143726,1460.986400,52943.533300,353.069400
std,0.000000e+00,47.783439,25.212207,0.828656,0.945457,1.067764,1.141805,1.203188,273.305024,0.0,1.666183,162.577763,2.013998e+18,0.388774,1788.778371,1511.150651,206.298149
min,1.237650e+18,8.235100,-5.382632,12.988970,12.799550,12.431600,11.947210,11.610410,308.000000,301.0,1.000000,11.000000,2.995780e+17,-0.004136,266.000000,51578.000000,1.000000
25%,1.237650e+18,157.370946,-0.539035,18.178035,16.815100,16.173333,15.853705,15.618285,752.000000,301.0,2.000000,184.000000,3.389248e+17,0.000081,301.000000,51900.000000,186.750000
50%,1.237650e+18,180.394514,0.404166,18.853095,17.495135,16.858770,16.554985,16.389945,756.000000,301.0,4.000000,299.000000,4.966580e+17,0.042591,441.000000,51997.000000,351.000000
75%,1.237650e+18,201.547279,35.649397,19.259232,18.010145,17.512675,17.258550,17.141447,1331.000000,301.0,5.000000,414.000000,2.881300e+18,0.092579,2559.000000,54468.000000,510.000000
max,1.237650e+18,260.884382,68.542265,19.599900,19.918970,24.802040,28.179630,22.833060,1412.000000,301.0,6.000000,768.000000,9.468830e+18,5.353854,8410.000000,57481.000000,1000.000000


In [4]:
sdss_df.drop(['objid', 'run', 'rerun', 'camcol', 'field', 'specobjid'], axis=1, inplace=True)
sdss_df.head(1)

,ra,dec,u,g,r,i,z,class,redshift,plate,mjd,fiberid
0,183.531326,0.089693,19.47406,17.0424,15.94699,15.50342,15.22531,STAR,-0.000009,3306,54922,491


## Feature Engineering

### u, g, r, i, z

We will now reduce the amount of dimensions by replacing the different bands 'u', 'g', 'r', 'i' and 'z' by a linear combination with only 3 dimensions using **Principal Component Analysis**.

**Principal Component Analysis:**

n observations with p features can be interpreted as n points in a p-dimensional space. PCA aims to project this space into a q-dimensional subspace (with q<p) with as little information loss as possible. 

It does so by finding the q directions in which the n points vary the most (--> the principal components). It then projects the original data points into the q-dimensional subspace. PCA returns a n x q dimensional matrix. 

Using PCA on our data will decrease the amount of operations during training and testing.

In [5]:
sdss_df_fe = sdss_df

# encode class labels to integers
le = LabelEncoder()
y_encoded = le.fit_transform(sdss_df_fe['class'])
sdss_df_fe['class'] = y_encoded

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

,ra,dec,class,redshift,plate,mjd,fiberid,PCA_1,PCA_2,PCA_3
0,183.531326,0.089693,2,-0.000009,3306,54922,491,-1.507202,-1.377293,-0.265119
1,183.598370,0.135285,2,-0.000055,323,51615,541,-0.195758,-0.028410,-0.155695
2,183.680207,0.126185,0,0.123111,287,52023,513,1.297604,-0.590023,0.140338
3,183.870529,0.049911,2,-0.000111,3306,54922,510,-1.446117,0.566685,-0.009272
4,183.883288,0.102557,2,0.000590,3306,54922,512,-0.849271,1.287505,-0.397689


## Machine Learning Models - Training

#### Feature Scaling

We will now train different models on this dataset. 

Scaling all values to be within the (0, 1) interval will reduce the distortion due to exceptionally high values and make some algorithms converge faster.

In [6]:
scaler = MinMaxScaler()
sdss = scaler.fit_transform(sdss_df_fe.drop('class', axis=1))

We will  split the data into a training and a test part. The models will be trained on the training data set and tested on the test data set

In [7]:
X_train, X_test, y_train, y_test = train_test_split(sdss, sdss_df_fe['class'], test_size=0.33)

In [8]:
models = {
    "K Nearest Neighbors Classifier": KNeighborsClassifier(),
    "Gaussian Naive Bayes Classifier": GaussianNB(),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=10),
    "Random Forest Classifier x100": RandomForestClassifier(n_estimators=100),
    "Support Vector Machine Classifier": SVC(),
}

In [9]:
# save these variables as artifacts because they will be used again and again for different experiments
lineapy.save(X_train, "X_train")
lineapy.save(X_test, "X_test")
lineapy.save(y_train, "y_train")
lineapy.save(y_test, "y_test")
lineapy.save(models, "models");

Consider a typical model training below using one of the models in our dictionary

In [10]:

modelname = "K Nearest Neighbors Classifier"
model = models[modelname]

model.fit(X_train, y_train)
preds = model.predict(X_test)
# model accuracies
acc = (preds == y_test).sum().astype(float) / len(preds)*100

# Alternately, we can perform k-fold cross validation. 
# We do this to get a more realistic result by testing the performance for 
# 10 different train and test datasets and averaging the results. Cross validation 
# ensures that the above result is not arbitary and gives a more reliable performance check.
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=10, scoring = "accuracy")
mean = scores.mean()
std = scores.std()

In [11]:
lineapy.save(acc, "accuracy")
lineapy.save(scores,"cvscores")
lineapy.save(mean,"cvmean")
lineapy.save(std,"cvstd");

In [12]:
repeat_eval_helper = lineapy.get_function(["accuracy","cvscores","cvmean","cvstd"], input_parameters=["modelname"], reuse_pre_computed_artifacts=["X_train", "X_test", "y_train", "y_test", "models"])

In [13]:
# NBVAL_IGNORE_OUTPUT

print(f"------------Random Forest Classifier------------")
xgboost_results = repeat_eval_helper("Random Forest Classifier")

print(f"Model accuracy: {xgboost_results['accuracy']}")
print(f"K-fold cv score: {xgboost_results['cvscores']}")
print(f"K-fold cv mean:  {xgboost_results['cvmean']}")
print(f"K-fold cv std:  {xgboost_results['cvstd']}")


print(f"------------Support Vector Machine Classifier------------")
svm_results = repeat_eval_helper("Support Vector Machine Classifier")

print(f"Model accuracy: {svm_results['accuracy']}")
print(f"K-fold cv score: {svm_results['cvscores']}")
print(f"K-fold cv mean:  {svm_results['cvmean']}")
print(f"K-fold cv std:  {svm_results['cvstd']}")


------------Random Forest Classifier------------
Model accuracy: 98.75757575757575
K-fold cv score: [0.99253731 0.99402985 0.99402985 0.99253731 0.99253731 0.9880597
 0.99402985 0.98955224 0.98955224 0.99104478]
K-fold cv mean:  0.9917910447761195
K-fold cv std:  0.0020300702251844216
------------Support Vector Machine Classifier------------
Model accuracy: 94.36363636363636
K-fold cv score: [0.94179104 0.96716418 0.96567164 0.94626866 0.94477612 0.9358209
 0.94179104 0.94477612 0.94776119 0.93731343]
K-fold cv mean:  0.9473134328358208
K-fold cv std:  0.010189778620118385


We can see that both Random Forest Classifier has a better performance than Support Vector Machines without having to re-do all the steps required for the evaluation.


### Summary

We trained different machine learning models to solve this classification problems. Scikit-Learn's Random Forest Classifier performed the best.

As Random Forest Classifier showed a little higher accuracy in most of the tests, we will continue only with this classifier.